# Tutorials

On this page, you will find several applicational examples on how Luna can be used to solve real-world problems.

## Beginner: Shipping wholesale products across Germany

Imagine the task of efficiently coordinating the shipment of wholesale products across Germany. You have retailers in the seven largest German cities and aim to deliver all your products at once while taking the shortest route possible to minimize transportation costs.

In technical terms, this challenge is known as the **Traveling Salesperson Problem (TSP)**, one of the most extensively studied optimization problems. It finds applications in various fields like planning, logistics, DNA sequencing, and even astronomy through slightly modified problem formulations.

Luna already allows you to create and solve a TSP without any manual work. You can visit our page on our preimplemented [use cases](https://docs.aqarios.com/use-cases) to find out more about this problem.

In simple terms, a TSP is fully defined by a weighted graph. Each node represents a city and each edge the distance between two cities.

### Creating a TSP instance

To create such a graph for a TSP, you can provide Luna with either a nested dictionary or a graph created using `networkx`. For simplicity and improved code readability, let's go with the latter approach.

In [ ]:
import networkx as nx

# Create an empty graph using networkx
graph = nx.Graph()

# Define the cities we want to visit
cities = ['Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt', 'Stuttgart', 'Düsseldorf']

# Add the cities as nodes in our graph
graph.add_nodes_from(cities)

# Define the distances between each city
edges = [('Berlin', 'Hamburg', 289), 
         ('Berlin', 'Munich', 586), 
         ('Berlin', 'Cologne', 574), 
         ('Berlin', 'Frankfurt', 567), 
         ('Berlin', 'Stuttgart', 634), 
         ('Berlin', 'Düsseldorf', 565), 
         ('Hamburg', 'Munich', 796), 
         ('Hamburg', 'Cologne', 425), 
         ('Hamburg', 'Frankfurt', 493), 
         ('Hamburg', 'Stuttgart', 656), 
         ('Hamburg', 'Düsseldorf', 401), 
         ('Munich', 'Cologne', 589), 
         ('Munich', 'Frankfurt', 392), 
         ('Munich', 'Stuttgart', 233), 
         ('Munich', 'Düsseldorf', 632), 
         ('Cologne', 'Frankfurt', 190), 
         ('Cologne', 'Stuttgart', 377), 
         ('Cologne', 'Düsseldorf', 42), 
         ('Frankfurt', 'Stuttgart', 205), 
         ('Frankfurt', 'Düsseldorf', 257), 
         ('Stuttgart', 'Düsseldorf', 416)]

# Add the distances as the edges in our graph
graph.add_weighted_edges_from(edges)

With this, we have defined our graph and, as a result, have already provided all the necessary specifications for a TSP. However, to actually transform this graph into a problem solvable using algorithms, you would typically need a mathematical model that enables communication with these algorithms designed to tackle optimization problems. The good news is that we have already taken care of implementing this mathematical model for you, so you don't need to handle it yourself.

Now, you can proceed to send our graph to Luna and generate a TSP instance.

In [ ]:
# Load the luna package
from luna import Luna

# Create a Luna object and set your credentials (more info on our Get Started page)
luna = Luna(credentials="credentials.json")

# Convert the networkx graph to a digestible format for Luna
graph_data = nx.to_dict_of_dicts(graph)

# Define that we want to create a TSP with the corresponding graph data
tsp_instance =  {
    "name": "TSP",
    "graph": graph_data
}

# Create a TSP instance and retrieve the corresponding ID
tsp_id = luna.create_optimization(instance=tsp_instance)

This covers all that is needed to create a TSP instance using Luna. We can now proceed to the step of selecting an appropriate algorithm to solve our specific problem.

### Choosing an Algorithm

Within Luna, you have access to a diverse range of pre-implemented algorithms, which are designed to solve various types of optimization problems. You can either manually choose an algorithm that best fits your needs or take advantage of our Recommendation Engine. This engine can provide you with a recommended algorithm, along with suggestions for the most suitable hardware to achieve optimal results.

Please note that the current version of our recommendation engine focuses on selecting from our available classical algorithms. However, we are actively working on introducing quantum algorithms in future versions.

Accessing the recommendation engine is straightforward – it's an integral part of LunaSolve, our dedicated service designed to efficiently solve real-world optimization challenges.

In [ ]:
# Create a LunaSolve session
ls = luna.start_LunaSolve()

To receive a recommendation, all you need to do is instruct LunaSolve to provide you with one. You'll need to supply the ID of the TSP instance we previously created.

In [ ]:
# Query the recommendation engine with our TSP instance
recommendation = ls.recommend(id=tsp_id)

With the recommendation in hand, you can now proceed directly to solving the problem using the suggested algorithm. If you're curious about the details of the recommendation, you can take a look before moving forward.

In [ ]:
print(recommendation)

**TODO: Pretty-Print** {'solver': 'tabu_search', 'solver_params': {'num_reads': 1}}

This will display all the relevant information about the chosen algorithm and its corresponding parameters. However, it's not necessary for you to delve into these details in order to simply apply the algorithm, if you prefer not to.

### Solving the Problem Instance

With our TSP instance created and a recommended algorithm obtained, we can now proceed to solve the problem. Thanks to Luna's interface, this can be achieved with just a single line of code.

In [ ]:
# Solve the TSP instance using our recommended algorithm and retrieve a solution url
solution_url = ls.solve(optimization_id=tsp_id, solver=recommendation)

Since certain algorithms, particularly those executed on quantum hardware, may require some time for completion, we have designed the process to be asynchronous. When you request a solution, Luna will manage all the necessary steps in the background. You're free to continue with other tasks while the computation is underway, returning at your convenience when it's finished.

While the raw output generated by these algorithms might not be immediately intuitive, especially if you're not well-versed in optimization methods, Luna offers a solution. We provide the capability to translate this raw output back into the context of your initial problem formulation. In the case of a TSP, this translation could simply involve generating a list of cities in the order they should be visited.

In [ ]:
# After the execution of your algorithm has been finished, retrieve your transformed solution
solution = ls.get_solution(solution_url, transform=True)

# Print the solution and check the order in which we should visit the cities
print(solution)

**TODO: Pretty-Print**
[['Hamburg'],
 ['Berlin'],
 ['Munich'],
 ['Stuttgart'],
 ['Frankfurt'],
 ['Cologne'],
 ['Düsseldorf']]

And with that, we've completed the process. As you've observed, Luna efficiently guides you through all the essential steps required to solve your optimization problems. Even if you're not well-versed in quantum computing or optimization techniques, you can seamlessly navigate the process. However, if you're interested in delving deeper into each step and customizing the process, you can explore our intermediate or experts tutorials.

## Intermediate: Maximizing Portfolio Value

Imagine you're interested in creating a portfolio of assets from the S&P500 index. Your primary goal is to ensure the security of your investment by minimizing the risk of potential losses. At the same time, you aim to achieve a certain minimum portfolio value to make the investment worthwhile. Additionally, you have a specific number of assets in mind that you intend to include in your portfolio.

As you explore our collection of pre-implemented use cases, you discover that we've already developed a solution tailored to your requirements. To define the problem, you need historical data on the returns of the candidate assets over a specified time period.

Let's assume you've prepared a CSV file containing the assets from the S&P500 index that you're considering for purchase. In this example, let's consider a scenario where you want to create a portfolio consisting of 2 out of 5 assets from the S&P500 index and achieve a target return higher or equal to the 75% quartile of all returns. Your data might look something like this:

In [ ]:
# You can use pandas to read a CSV file
import pandas as pd

# Read the CSV file containing the assets you want to choose from
# TODO: Change delimiter in CSV file
assets = pd.read_csv('portfolio.csv', sep=";")

# Print the assets
print(assets)

# **TODO: add CSV markdown from print statement

### Creating a Portfolio Optimization Instance

To optimize the selection of assets for your portfolio, you can use our LunaSolve service. To do this, you need to create a portfolio optimization (PO) instance, which requires your data to be formatted in a way suitable for optimization algorithms. In the case of portfolio optimization, we've already established a mathematical format for this purpose. Specifically, we use a Quadratic Unconstrained Binary Optimization (QUBO) problem formulation. 

For more details on this formulation, you can refer to our [portfolio optimization use case](www.docs.aqarios.com/use-cases/#portfolio-optimization). There, you'll find information about the problem's mathematical representation. Note that we require a two-dimensional list as input data. Therefore, let's start by transforming your CSV file into the correct format: a list of lists containing the time series of returns per asset. 

In [ ]:
# Numpy allows for easier modification of the data
import numpy as np

# Remove the Date column from the data
returns = assets.iloc[:, 1:]

# Convert the data to a numpy array
returns = np.array(returns)

# Switch axes to retrieve the time series of each asset
returns = np.transpose(returns)

# Convert the returns to a list of lists
returns = returns_np.tolist()

Now we only need to define our target return and the number of assets we want to buy.

In [ ]:
# Define the minimum return you want to achieve, at least the 75% quartile of all returns
target_return = np.percentile(returns, 75)

# Define the number of assets you want to buy
n_assets = 2

Next, we can send your portfolio data to Luna and create the optimization instance.

In [ ]:
# Load the luna package
from luna import Luna

# Create a Luna object and set your credentials (more info on our Get Started page)
luna = Luna(credentials="credentials.json")

# Define that we want to create an instance of a PO with the corresponding data
po_instance =  {
    "name": "PO",
    'returns': returns,
    'R': target_return,
    'n': n_assets,
}

# Create a PO instance and retrieve the corresponding ID
po_id = luna.create_optimization(instance=po_instance)

This completes the process of creating a PO instance through Luna. We are now ready to move on to the selection of an algorithm to solve our optimization problem.

### Solving the problem instance

Luna offers a diverse range of pre-implemented algorithms that are designed to tackle various types of optimization problems. You have the option to manually choose an algorithm that suits your needs or use our Recommendation Engine to suggest the most suitable algorithm. In this tutorial, let's assume that you'd like to apply a solver developed by us — the hybrid [QAGA+](link todo) solver. This evolutionary algorithm combines classical hardware with Quantum Annealers from D-Wave Systems.

QAGA+ is a highly configurable algorithm with numerous hyperparameters that can be adjusted to enhance performance across different optimization problems. For simplicity, we will only configure three parameters in this tutorial:

- `p_size`: The population size, determining how many individuals remain at the end of each iteration.
- `mut_rate`: The mutation rate, indicating the likelihood of each individual undergoing mutation.
- `rec_rate`: The recommendation rate, determining the number of mates each individual is paired with in each iteration.

In practical scenarios, we recommend reviewing the complete list of available parameters and setting them based on your specific needs when selecting your algorithm. While default parameters are configured to function effectively across a wide range of applications, they might not always yield optimal performance.

Now, let's proceed with the necessary steps to solve the portfolio optimization problem using QAGA+. Firstly, we need to establish our access token for the hardware provided by D-Wave Systems (this is only necessary once). If you don't possess such a token, you can obtain one directly from D-Wave, or you can reach out to us for questions about the process.

In [ ]:
# Set your token to access the Quantum Annealer from D-Wave Systems
luna.set_qpu_token(provider='dwave', token='<QPUTOKEN>')

Following that, we can set the solver and its corresponding parameters to proceed with solving the optimization problem using LunaSolve.

In [ ]:
# Define QAGA+ as our solver
solver = 'QAGA+'

# Define the three parameters for QAGA+ we want to set
params = {
    'p_size': 40,
    'mut_rate': 0.3,
    'rec_rate': 2
}

# Create a LunaSolve session
ls = luna.start_LunaSolve()

# Solve the PO instance using the QAGA+ algorithms and retrieve a solution url
solution_url = ls.solve(optimization_id=po_id, solver=solver, parameters=params)

Given that QAGA+ involves computations on real quantum hardware from D-Wave Systems, the execution might require some time. To ensure that you don't have to wait for the code to finish executing, we have designed the process to be asynchronous. This means that when you request a solution, LunaSolve takes care of all the necessary steps in the background, allowing you the freedom to engage in other tasks while the solution is being computed. You can simply return when the execution is complete.

There are two ways to retrieve your solution:

1. **Raw Solution:** This corresponds to the solution derived from the mathematical model of the problem. In our scenario, where we're dealing with QUBOs, this will be a binary vector.

2. **Transformed Solution:** In this case, the raw solution, represented in mathematical format, has been translated back into the initial problem domain. For our portfolio optimization, this means obtaining the list of assets that should be purchased.

To begin, let's examine the raw solution.

In [ ]:
# After the execution of your algorithm has been finished, retrieve your raw solution
solution = ls.get_solution(solution_url)

# Print the raw solution
print(solution)

However, the raw output provided by these algorithms might not be particularly informative, especially when working with our pre-implemented problem formulations. Instead, let's focus on the transformed solution, which is more aligned with the initial problem formulation and easier to interpret in the context of our use case.

In [ ]:
# Alternatively, retrieve the transformed solution
solution = ls.get_solution(solution_url, transform=True)

# Print the transformed solution and check which assets we should buy
print(solution)

Luna only knows about the indices of each asset and not the names, but you can easily access the names in your original data.

In [ ]:
# Get the names of your initial assets
asset_names = assets.columns.tolist()[1:]

# Retrieve the names of the assets you should buy from the solution
assets_to_buy = [asset_names[i] for i in solution['solution']]

# Print the assets to buy
print(assets_to_buy)

And there you have it – we've reached the end of this tutorial! With Luna, you have the capability to efficiently solve optimization problems and apply a wide range of algorithms. You're also free to configure each step in the process according to your needs. If you're looking for more inspiration, there's a whole library of [use cases](https://docs.aqarios.com/use-cases) and [use cases](https://docs.aqarios.com/solvers-and-algorithms) available for exploration. Alternatively, you're welcome to explore our Expert Tutorials, which provide more in-depth insights. For instance, you can delve into solving your own unique optimization problems using Luna's capabilities.

## Expert: Solving arbitrary QUBOs using QAOA

Luna offers more than just its use case library – it's also a versatile tool for solving custom optimization problems you might encounter. In this tutorial, we'll explore a scenario where you've developed a proprietary transformation for a specific use case that you want to keep in-house. This transformation helps convert your optimization problem into the QUBO format.

Luna provides all its features in such cases too. You can effortlessly upload your custom optimization problem to Luna and store it in your account.

In [ ]:
# Load the luna package
from luna import Luna

# Create a Luna object and set your credentials (more info on our Get Started page)
luna = Luna(credentials="credentials.json")

# Retrieve or define your custom QUBO. It has to be a matrix represented as a list of lists
qubo = [
    [4, 0, 0, 0, -2],
    [0, -2, 0, 0, 0],
    [0, 0, 6, -3, 0],
    [0, 0, -3, 2, 0],
    [-2, 0, 0, 0, 5]
]

# Send and store the QUBO and retrieve the corresponding ID
qubo_id = luna.create_qubo(instance=tsp_instance)

From this point onward, you have the option to select one of our pre-implemented QUBO solver algorithms. You can dedice to manually choose your preferred algorithm or use our recommendation engine. It's important to note that while the recommendation engine can work with raw QUBO formulations, the suggestions might be more accurate when dealing with use cases from our library due to the additional context they provide. Keep this in mind when using this feature.

In this case, let's assume that you want to solve your QUBO using the QAOA algorithm on one of IBM's free-to-use simulators. Specifically, we'll use the QASM Simulator for this tutorial.

In [ ]:
# Define QAOA as our solver
solver = 'QAGA+'

# Define the QASM Simulator as backend for QAOA
params = {
    'backend': 'qasm_simulator'
}

# Create a LunaSolve session
ls = luna.start_LunaSolve()

# Solve your QUBO using our QAOA on the QASM Simulator from IBM and retrieve a solution url
solution_url = ls.solve_qubo(qubo_ids=qubo_id, solver=solver, parameters=params)

Since IBM's simulators run on their servers rather than ours, the execution might take some time. To prevent you from waiting for the code to complete, we've designed the process to be asynchronous. When you request a solution, LunaSolve handles all the required steps in the background. This way, you can continue with other tasks while the solution is being computed. You just need to return when the execution is finished.

After the algorithms have completed their run, you can easily retrieve the solution for your QUBO. This solution will be a binary vector, indicating the settings of the variables.

In [ ]:
# After the execution of your algorithm has finished, retrieve your solution
solution = ls.get_solution(solution_url)

# Print the solution - a binary vector
print(solution)

Now, you can take the results and perform post-processing in your local environment.

## Expert: Running Qiskit Programs on IBM Quantum

While Luna primarily focuses on optimization services, its capabilities extend beyond that domain. With LunaQ, we provide a framework that greatly simplifies the process of accessing various quantum hardware providers. You can use this framework to execute algorithms from our use case library or deploy your custom algorithms.

In this tutorial, let's imagine you have a Qiskit program locally that you wish to run on IBM's servers. Through LunaQ, we offer the capability to submit arbitrary programs to IBM Quantum.

In [ ]:
# Load the luna package
from luna import Luna

# Create a Luna object and set your credentials (more info on our Get Started page)
luna = Luna(credentials="credentials.json")

# Create a LunaQ session
lq = luna_start_LunaQ()

To execute your custom Qiskit program, you need to provide a Python file containing the Qiskit code and a JSON file for the corresponding metadata. In this tutorial, let's consider a scenario where you aim to measure a Bell state in 5 iterations. For flexibility, you've kept the number of iterations as a parameter within your script. Here's what your Qiskit program might look like:

In [ ]:
from typing import Any

from qiskit_ibm_runtime.program import UserMessenger, ProgramBackend
from qiskit import QuantumCircuit, QuantumRegister, transpile


def circuit(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr)

    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.measure_all()

    return transpile(qc, backend)

def main(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs) -> Any:
    iterations = kwargs.pop("iterations", 5)
    counts = {"00": 0, "11": 0}
    for it in range(iterations):
        qc = circuit(backend, user_messenger)
        result = backend.run(qc).result()
        counts_it = result.get_counts()
        counts["00"] += counts_it["00"]
        counts["11"] += counts_it["11"]

    return counts

Now, let's assume you intend to run this program on IBM's Matrix Product State Simulator.

In [ ]:
# The path to the file that contains your Qiskit program
program_path = "program.py"

# The path to the file that contains the metadata of your Qiskit program
metadata = "program.json"

# The inputs to your Qiskit program, here the number of iterations of running your circuit
inputs = {"iterations": 10}

# Define the MPS Simulator as backend for your program
params = {"backend_name": "simulator_mps"}

With just a single line of code, you can execute this program through Luna on IBM's server.

In [ ]:
circuit_id = lq.send_qiskit_program(program_path=program_path, metadata=metadata, inputs=inputs, options=options)

Since IBM's simulators run on their servers rather than ours, the execution might take some time. To prevent you from waiting for the code to complete, we've designed the process to be asynchronous. When you request a solution, LunaSolve handles all the required steps in the background. This way, you can continue with other tasks while the solution is being computed. You just need to return when the execution is finished.

After the program has finished running, you can easily retrieve the output through Luna again.

In [ ]:
# Retrieve the results of your Qiskit program
result = lq.get_qiskit_results(circuit_id)

# Print the results
print(result)

This service is based on the Qiskit runtime service to upload runtime programs, so you can refer to [IBM's documentation](https://qiskit.org/ecosystem/ibm-runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.upload_program.html) for further details.